In [1]:
import uhd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [2]:

usrp = uhd.usrp.MultiUSRP()


center_freq = 2.415e9# Hz
sample_rate = 56e6 # Hz twice the BW
nr_batches=10000 #1000 batches of 20,000 samples almost 1s
batch = 21000
THRESHOLD = 0.01
gain = 40# dB

usrp.set_rx_rate(sample_rate, 0)
usrp.set_rx_freq(uhd.libpyuhd.types.tune_request(center_freq), 0)
usrp.set_rx_gain(gain, 0)
usrp.set_rx_agc(False, 0)

[INFO] [UHD] linux; GNU C++ version 9.4.0; Boost_107100; UHD_3.15.0.HEAD-0-gaea0e2de
[INFO] [B200] Detected Device: B210
[INFO] [B200] Operating over USB 3.
[INFO] [B200] Initialize CODEC control...
[INFO] [B200] Initialize Radio control...
[INFO] [B200] Performing register loopback test... 
[INFO] [B200] Register loopback test passed
[INFO] [B200] Performing register loopback test... 
[INFO] [B200] Register loopback test passed
[INFO] [B200] Setting master clock rate selection to 'automatic'.
[INFO] [B200] Asking for clock rate 16.000000 MHz... 
[INFO] [B200] Actually got clock rate 16.000000 MHz.
[INFO] [B200] Asking for clock rate 56.000000 MHz... 
[INFO] [B200] Actually got clock rate 56.000000 MHz.


In [3]:

# Set up the stream and receive buffer
st_args = uhd.usrp.StreamArgs("fc32", "sc16")
st_args.channels = [0]
metadata = uhd.types.RXMetadata()
streamer = usrp.get_rx_stream(st_args)
recv_buffer = np.zeros((1, batch), dtype=np.complex64)

# Start Stream
stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.start_cont)
stream_cmd.stream_now = True
streamer.issue_stream_cmd(stream_cmd)

In [4]:
# Receive Samples
import numpy as np
import time
file = "ESP32_Adv_sampling_rate_"+str(round(sample_rate/1e6,2))+"_gain_"+str(gain)+"_fc_"+str(round(center_freq/1e6,3))+".iq"
f = open(file,"wb")

# samples = np.zeros(batch, dtype=np.complex64)


# print(recordingTime <time.time() - start)
for i in range(nr_batches):#   recordingTime >time.time() - start:
    streamer.recv(recv_buffer, metadata)
    np.zeros(1).tofile(f)
    recv_buffer[0].tofile(f)

# Stop Stream
stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.stop_cont)
streamer.issue_stream_cmd(stream_cmd)
f.close()
# we will only take the FFT of the first 1024 samples, see text below


OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

In [5]:
samples = np.fromfile(file, np.complex64) # Read in file.  We have to tell it what format it is
zeros = np.abs(samples)<THRESHOLD
samples[zeros] = 0

In [6]:
from itertools import groupby
from operator import itemgetter

def frameFinder(samples):
    test_list = np.nonzero(samples)
    framesIndex = []
    for k, g in groupby(enumerate(test_list[0]), lambda ix: ix[0]-ix[1]):
        temp = list(map(itemgetter(1), g))
        if len(temp)< 1000:
            continue
        framesIndex.append([temp[0],temp[-1]])
    return np.array(framesIndex)

def zeroRemover(file,framesIndex):
    f = open(file,"wb")
    for i,j in framesIndex:
        frame = samples[i:j]
        frame.tofile(f)
        np.zeros(2000,dtype=np.complex64).tofile(f)
    f.close()


zeroRemover(file, frameFinder(samples))     